In [1]:
import os 
os.chdir("./all")


In [2]:
!ls

sample_submission.csv  test.zip  train1  train.zip


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt 
#plt.rcParams['figure.figsize'] = [16, 10]
#plt.rcParams['font.size'] = 16
import os
#import seaborn as sns
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.datasets import load_files 
from keras.utils import np_utils
from glob import glob


Using TensorFlow backend.


In [4]:
# define function to load datasets
def load_dataset(path):
    data = load_files(path)
    X = np.array(data['filenames'])
    ip_files = np.array([i for i in X])
    ip_targets = data['target']
    return ip_files, ip_targets

# load datasets
files, targets = load_dataset('./train1')


# load list of plant names
# first 15 characters are of path 
plant_names = [i[9:-1] for i in sorted(glob("./train1/*/"))]

# print statistics about the dataset
print('There are %d total categories.' % len(plant_names))
print('There are %s total images.\n' % len(files))


There are 12 total categories.
There are 4750 total images.



In [5]:
print (targets[0])

9


In [6]:
# How many images per category
for name in plant_names:
    print('{} {} images'.format(name, len(os.listdir(os.path.join("./train1", name)))))

Black-grass 263 images
Charlock 390 images
Cleavers 287 images
Common Chickweed 611 images
Common wheat 221 images
Fat Hen 475 images
Loose Silky-bent 654 images
Maize 221 images
Scentless Mayweed 516 images
Shepherds Purse 231 images
Small-flowered Cranesbill 496 images
Sugar beet 385 images


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(files, targets, test_size=0.2, random_state=42)


In [8]:
X_train[0]

'./train1/Fat Hen/45c197012.png'

In [9]:
!ls

sample_submission.csv  test.zip  train1  train.zip


In [14]:
!mkdir test train 
!ls

sample_submission.csv  test  test.zip  train  train1  train.zip


In [15]:
for i in ['test', 'train']:
    for j in plant_names:
        path = "./"+i+"/"+j+"/"
        os.mkdir(path)

In [16]:
for source in [(X_train,y_train,"train"),(X_test,y_test,"test")]:
    for counter, old_loc in enumerate(source[0]):
        name = old_loc.split("/")[-1]
        new_loc = '"' + "./" +source[2] + "/" + plant_names[source[1][counter]] + "/" + name + '"'
        #print (new_loc)
        os.system('cp '+ '"'+old_loc+'"' + ' ' + new_loc)


In [19]:
# define function to load datasets
def load_dataset(path):
    data = load_files(path)
    ip_files = np.array(data['filenames'])
    ip_targets = np_utils.to_categorical(np.array(data['target']),12 )
    return ip_files, ip_targets

# load datasets
X_train, y_train = load_dataset('./train')
X_valid, y_valid = load_dataset('./test')


# load list of plant names
# first 15 characters are of pat

# print statistics about the dataset
print('There are %d total categories.' % len(plant_names))
print ("Total Images are 4750")
print('There are %s total training images.\n' % len(X_train))

print('There are %s total testing images.\n' % len(X_test))

There are 12 total categories.
Total Images are 4750
There are 3800 total training images.

There are 950 total testing images.



In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categoriacl',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)